# AWS Bedrock Access Validation - Governance First

## Purpose
This notebook validates AWS Bedrock access with a governance-first approach, capturing telemetry required for:
- Cost attribution and FinOps
- Compliance auditing
- Provider SLA monitoring
- Capacity planning

## Prerequisites
- AWS credentials configured (environment variables, ~/.aws/credentials, or IAM role)
- Bedrock access enabled in your AWS account
- Bedrock model access granted (specifically Meta Llama models)

## Governance Principles
1. **Identity verification** - Know who/what is making API calls
2. **Telemetry capture** - All invocations must be measurable
3. **Failure classification** - Distinguish billing, permission, and availability issues
4. **Audit readiness** - Structured logging for compliance reviews

In [9]:
# Core imports
import boto3
import json
import time
from datetime import datetime, timezone
from typing import Dict, Any, Optional
import pandas as pd

## A. Authentication & Identity

### IAM Authentication Model
AWS Bedrock uses IAM (Identity and Access Management) for authentication and authorization:
- **Authentication**: Verified via AWS credentials (access keys, IAM roles, or temporary credentials)
- **Authorization**: Controlled by IAM policies attached to the identity
- **Required permissions**: `bedrock:InvokeModel`, `bedrock:ListFoundationModels`

### Identity Validation
We use AWS STS (Security Token Service) to validate the calling identity before any Bedrock operations.
This ensures:
- Credentials are valid
- We know which principal is being charged
- Audit trails are attributable

In [2]:
# Validate calling identity using STS
def validate_identity() -> Dict[str, Any]:
    """
    Validate AWS credentials and return calling identity information.
    Critical for governance: establishes accountability for API usage.
    """
    try:
        sts_client = boto3.client('sts')
        identity = sts_client.get_caller_identity()
        
        print("✓ AWS Identity Validated")
        print(f"  Account: {identity['Account']}")
        print(f"  ARN: {identity['Arn']}")
        print(f"  UserId: {identity['UserId']}")
        
        return identity
    except Exception as e:
        print(f"✗ Identity Validation Failed: {str(e)}")
        raise

# Execute validation
identity = validate_identity()

✓ AWS Identity Validated
  Account: 108839616732
  ARN: arn:aws:iam::108839616732:user/Hamdan
  UserId: AIDARSV2YYDOJNXPVBBFF


## B. API / Runtime Validation

### Bedrock Runtime Architecture
- **Control Plane**: `bedrock` client - lists models, manages access
- **Data Plane**: `bedrock-runtime` client - invokes models

### Model Selection
We target Meta Llama models (Instruct variants) for:
- Open-source governance alignment
- Predictable licensing
- Strong instruction-following capabilities

In [16]:
# List available Bedrock foundation models
def list_available_models(provider_filter: Optional[str] = "meta") -> list:
    """
    Enumerate available models in Bedrock.
    Governance value: Verify entitlements and discover new models.
    """
    try:
        bedrock_client = boto3.client('bedrock', region_name='us-east-1')
        response = bedrock_client.list_foundation_models()
        
        models = response.get('modelSummaries', [])
        
        if provider_filter:
            models = [m for m in models if provider_filter.lower() in m.get('providerName', '').lower()]
        
        print(f"✓ Found {len(models)} models" + (f" (filtered by: {provider_filter})" if provider_filter else ""))
        
        for model in models[:5]:  # Show first 5
            print(f"  - {model['modelId']}")
            print(f"    Provider: {model.get('providerName', 'N/A')}")
            print(f"    Inference: {', '.join(model.get('inferenceTypesSupported', []))}")
        
        if len(models) > 5:
            print(f"  ... and {len(models) - 5} more")
        
        return models
    except Exception as e:
        print(f"✗ Model Listing Failed: {str(e)}")
        raise

# Execute model discovery
df = pd.DataFrame(list_available_models(provider_filter="meta"))
df


✓ Found 11 models (filtered by: meta)
  - meta.llama3-8b-instruct-v1:0
    Provider: Meta
    Inference: ON_DEMAND
  - meta.llama3-70b-instruct-v1:0
    Provider: Meta
    Inference: ON_DEMAND
  - meta.llama3-1-8b-instruct-v1:0
    Provider: Meta
    Inference: INFERENCE_PROFILE
  - meta.llama3-1-70b-instruct-v1:0
    Provider: Meta
    Inference: INFERENCE_PROFILE
  - meta.llama3-2-11b-instruct-v1:0
    Provider: Meta
    Inference: INFERENCE_PROFILE
  ... and 6 more


,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
0,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama3-8b-instruct-v1:0,Llama 3 8B Instruct,Meta,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
1,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama3-70b-instruct-v1:0,Llama 3 70B Instruct,Meta,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
2,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama3-1-8b-instruct-v1:0,Llama 3.1 8B Instruct,Meta,[TEXT],[TEXT],True,[],[INFERENCE_PROFILE],{'status': 'ACTIVE'}
3,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama3-1-70b-instruct-v1:0,Llama 3.1 70B Instruct,Meta,[TEXT],[TEXT],True,[],[INFERENCE_PROFILE],{'status': 'ACTIVE'}
4,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama3-2-11b-instruct-v1:0,Llama 3.2 11B Instruct,Meta,"[TEXT, IMAGE]",[TEXT],True,[],[INFERENCE_PROFILE],{'status': 'ACTIVE'}
5,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama3-2-90b-instruct-v1:0,Llama 3.2 90B Instruct,Meta,"[TEXT, IMAGE]",[TEXT],True,[],[INFERENCE_PROFILE],{'status': 'ACTIVE'}
6,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama3-2-1b-instruct-v1:0,Llama 3.2 1B Instruct,Meta,[TEXT],[TEXT],True,[],[INFERENCE_PROFILE],{'status': 'ACTIVE'}
7,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama3-2-3b-instruct-v1:0,Llama 3.2 3B Instruct,Meta,[TEXT],[TEXT],True,[],[INFERENCE_PROFILE],{'status': 'ACTIVE'}
8,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama3-3-70b-instruct-v1:0,Llama 3.3 70B Instruct,Meta,[TEXT],[TEXT],True,[],[INFERENCE_PROFILE],{'status': 'ACTIVE'}
9,arn:aws:bedrock:us-east-1::foundation-model/me...,meta.llama4-scout-17b-instruct-v1:0,Llama 4 Scout 17B Instruct,Meta,"[TEXT, IMAGE]",[TEXT],True,[],[INFERENCE_PROFILE],{'status': 'ACTIVE'}


In [4]:
# Select a Meta Llama model for invocation
# Using Llama 3.1 8B Instruct as default (cost-effective, widely available)
MODEL_ID = "us.meta.llama3-2-1b-instruct-v1:0"  # Adjust based on your region/access

print(f"Selected Model: {MODEL_ID}")
print("Note: Adjust MODEL_ID if this model is unavailable in your account")

Selected Model: us.meta.llama3-2-1b-instruct-v1:0
Note: Adjust MODEL_ID if this model is unavailable in your account


## AI Model Metadata & Governance Tagging

Before provisioning or invoking any AI model, governance platforms require
explicit **model metadata and tagging** to enable cost attribution, policy
enforcement, and auditability.

Each model invocation is classified using the following governance tags:

- Provider (e.g., aws_bedrock, openai)
- Model ID (exact model identifier)
- Model Class (reasoning / general / lightweight)
- Risk Tier (low / medium / high)
- Cost Tier (budget / standard / premium)

These tags are captured at invocation time and form the basis for:
- FinOps cost allocation
- Policy enforcement
- Usage analytics
- Cross-cloud normalization


## C. Model Invocation with Governance Telemetry

### Why These Metrics Matter

| Metric | Governance Value |
|--------|------------------|
| **provider** | Vendor risk management, multi-cloud strategy |
| **model_id** | Cost allocation, performance benchmarking |
| **latency_ms** | SLA compliance, user experience tracking |
| **token_usage** | FinOps (primary cost driver), capacity planning |
| **timestamp** | Audit trails, usage pattern analysis |
| **failure_classification** | Operational health, cost avoidance (detect quota issues) |

### Token Usage Caveat
Not all Bedrock models return token counts in responses. This is a **provider limitation**, not a notebook issue.
For governance: Log as `null` when unavailable, track separately via CloudWatch if needed.

In [22]:
# Governance telemetry structure
class InvocationTelemetry:
    """
    Structured telemetry for AI governance.
    Designed for export to SIEM, data warehouse, or FinOps platforms.
    """
    def __init__(self):
        self.provider = None
        self.model_id = None
        self.latency_ms = None
        self.input_tokens = None
        self.output_tokens = None
        self.timestamp = None
        self.success = False
        self.failure_classification = None
        self.error_message = None
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            "provider": self.provider,
            "model_id": self.model_id,
            "latency_ms": self.latency_ms,
            "input_tokens": self.input_tokens,
            "output_tokens": self.output_tokens,
            "total_tokens": (self.input_tokens + self.output_tokens) if (self.input_tokens and self.output_tokens) else None,
            "timestamp": self.timestamp,
            "success": self.success,
            "failure_classification": self.failure_classification,
            "error_message": self.error_message
        }
    
    def log(self):
        """Print telemetry in audit-ready format"""
        print("\n" + "="*60)
        print("GOVERNANCE TELEMETRY")
        print("="*60)
        for key, value in self.to_dict().items():
            print(f"{key:.<30} {value}")
        print("="*60)

# Initialize telemetry collector
telemetry = InvocationTelemetry()

## D. Failure Handling & Classification

### Failure Categories
- **PERMISSION_DENIED**: IAM policy issues, model access not granted
- **THROTTLING**: Rate limits, quota exhaustion (critical for cost management)
- **VALIDATION_ERROR**: Malformed request (code issue, not governance issue)
- **SERVICE_ERROR**: Provider-side failures
- **UNKNOWN**: Unclassified errors (requires investigation)

### Governance Implications
- **Throttling** → Indicates capacity planning needed or unexpected usage spike
- **Permission errors** → Audit IAM policies, verify entitlements
- **Service errors** → Track for SLA compliance, consider multi-provider strategy

In [6]:
def classify_failure(error: Exception) -> str:
    """
    Classify failures for governance reporting.
    Maps AWS exceptions to governance-relevant categories.
    """
    error_str = str(error)
    error_type = type(error).__name__
    
    # AWS Bedrock specific error patterns
    if "AccessDeniedException" in error_type or "UnauthorizedException" in error_type:
        return "PERMISSION_DENIED"
    elif "ThrottlingException" in error_type or "TooManyRequestsException" in error_type:
        return "THROTTLING"
    elif "ValidationException" in error_type or "invalid" in error_str.lower():
        return "VALIDATION_ERROR"
    elif "ServiceException" in error_type or "InternalServerError" in error_type:
        return "SERVICE_ERROR"
    elif "ModelNotReadyException" in error_type:
        return "MODEL_UNAVAILABLE"
    else:
        return "UNKNOWN"

In [ ]:
def invoke_llama_with_governance(prompt: str, model_id: str) -> Dict[str, Any]:
    """
    Invoke Meta Llama model with full governance telemetry.
    
    Returns:
        dict: Contains 'response_text' and 'telemetry' keys
    """
    telemetry = InvocationTelemetry()
    telemetry.model_id = model_id
    telemetry.provider = "aws_bedrock"
    telemetry.timestamp = datetime.now(timezone.utc).isoformat()
    
    try:
        bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')
        
        # Construct request payload for Llama models
        request_body = {
            "prompt": prompt,
            "max_gen_len": 512,
            "temperature": 0.7,
            "top_p": 0.9
        }
        
        # Measure latency
        start_time = time.time()
        
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(request_body),
            contentType='application/json',
            accept='application/json'
        )
        
        end_time = time.time()
        telemetry.latency_ms = round((end_time - start_time) * 1000, 2)
        
        # Parse response
        response_body = json.loads(response['body'].read())
        
        # Extract response text
        response_text = response_body.get('generation', '')
        
        # Extract token usage (Meta Llama models may not always provide this)
        telemetry.input_tokens = response_body.get('prompt_token_count')
        telemetry.output_tokens = response_body.get('generation_token_count')
        
        # Handle missing token metrics (provider limitation)
        if telemetry.input_tokens is None:
            print("⚠ Warning: Input token count not provided by model (provider limitation)")
        if telemetry.output_tokens is None:
            print("⚠ Warning: Output token count not provided by model (provider limitation)")
        
        telemetry.success = True
        
        print("✓ Model Invocation Successful")
        print(f"  Latency: {telemetry.latency_ms}ms")
        print(f"  Response Length: {len(response_text)} characters")
        
        return {
            "response_text": response_text,
            "telemetry": telemetry
        }
        
    except Exception as e:
        telemetry.success = False
        telemetry.failure_classification = classify_failure(e)
        telemetry.error_message = str(e)
        
        print(f"✗ Model Invocation Failed")
        print(f"  Classification: {telemetry.failure_classification}")
        print(f"  Error: {str(e)}")
        
        # Re-raise for caller to handle, but telemetry is captured
        return {
            "response_text": None,
            "telemetry": telemetry,
            "error": str(e)
        }

# Execute invocation with governance
test_prompt = "Explain the concept of cloud governance in exactly two sentences."

result = invoke_llama_with_governance(
    prompt=test_prompt,
    model_id=MODEL_ID
)

# Display telemetry
result['telemetry'].log()

✓ Model Invocation Successful
  Latency: 5673.57ms
  Response Length: 2112 characters

GOVERNANCE TELEMETRY
provider...................... aws_bedrock
model_id...................... us.meta.llama3-2-1b-instruct-v1:0
latency_ms.................... 5673.57
input_tokens.................. 12
output_tokens................. 379
total_tokens.................. 391
timestamp..................... 2026-02-01T12:46:28.774984+00:00
success....................... True
failure_classification........ None
error_message................. None


In [8]:
# Display model response if successful
if result['response_text']:
    print("\n" + "="*60)
    print("MODEL RESPONSE")
    print("="*60)
    print(result['response_text'])
    print("="*60)
else:
    print("\n⚠ No response generated due to invocation failure")
    print(f"Error: {result.get('error', 'Unknown')}")


MODEL RESPONSE
 Cloud governance refers to the processes and practices that organizations use to manage and govern their cloud-based resources, including data, applications, and infrastructure, to ensure efficient and secure use. Effective cloud governance involves setting clear policies, monitoring and managing cloud resources, and collaborating with cloud providers to ensure compliance with organizational standards and regulatory requirements.


## OPEN AI


## Authentication & Identity Model — OpenAI

OpenAI uses a **static API key–based authentication model**.  
Unlike cloud-native providers (AWS/GCP), OpenAI does **not provide native IAM, roles, or identity federation**.

**Authentication Mechanism**
- API Key passed via environment variable
- No per-user or per-role identity context at the API level

**Governance Implications**
- API keys represent a shared identity and must be centrally managed
- No native RBAC or policy enforcement
- Requires an internal proxy or gateway layer for:
  - Key rotation
  - Per-user attribution
  - Access control enforcement

**Enterprise Consideration**
OpenAI is easy to onboard but introduces **higher governance risk** without additional platform controls.


In [19]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Under a silver moon, a gentle unicorn trotted through a sleeping meadow, curled up on a pillow of moss, and whispered goodnight to the stars.


## Governance Telemetry Mapping

Each OpenAI model invocation is captured using a **standardized governance telemetry schema**.

**Captured Signals**
- Provider: `openai`
- Model identifier
- Request latency (ms)
- Input and output token counts
- Invocation success or failure
- Error classification (if applicable)

**Why This Matters**
Structured telemetry enables:
- Cost attribution and FinOps analysis
- Cross-provider normalization
- Policy enforcement and auditability
- Anomaly detection and usage forecasting

This schema is intentionally aligned with telemetry collected from IAM-based providers (AWS/GCP).


In [ ]:
import os
import time
from datetime import datetime, timezone
from typing import Dict, Any

from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()


def invoke_openai_with_governance(
    prompt: str,
    model: str = "gpt-4o"
) -> Dict[str, Any]:
    """
    Invoke OpenAI with governance telemetry adaptation.
    Normalizes OpenAI metrics into our standard InvocationTelemetry format.
    """

    telemetry = InvocationTelemetry()
    telemetry.provider = "openai"
    telemetry.model_id = model
    telemetry.timestamp = datetime.now(timezone.utc).isoformat()

    try:
        # Ensure API key is present
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OPENAI_API_KEY not found in environment variables")

        client = OpenAI(api_key=api_key)

        start_time = time.time()

        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        

        end_time = time.time()
        telemetry.latency_ms = round((end_time - start_time) * 1000, 2)

        # Capture response text
        response_text = response.choices[0].message.content

        # Capture token usage
        if response.usage:
            telemetry.input_tokens = response.usage.prompt_tokens
            telemetry.output_tokens = response.usage.completion_tokens

        telemetry.success = True
        print(f"✓ OpenAI Invocation Successful ({model})")

        return {
            "response_text": response_text,
            "telemetry": telemetry
        }

    except Exception as e:
        telemetry.success = False
        telemetry.failure_classification = classify_failure(e)
        telemetry.error_message = str(e)

        print(f"✗ OpenAI Invocation Failed: {str(e)}")

        return {
            "response_text": None,
            "telemetry": telemetry
        }


# Execute Governance Check
openai_result = invoke_openai_with_governance(
    prompt="Explain zero-trust security in one sentence.",
    model="gpt-3.5-turbo"
)

# Log Telemetry
openai_result["telemetry"].log()


✓ OpenAI Invocation Successful (gpt-3.5-turbo)

GOVERNANCE TELEMETRY
provider...................... openai
model_id...................... gpt-3.5-turbo
latency_ms.................... 1916.02
input_tokens.................. 17
output_tokens................. 31
total_tokens.................. 48
timestamp..................... 2026-02-01T13:49:47.634102+00:00
success....................... True
failure_classification........ None
error_message................. None


## Governance Implications — OpenAI vs Cloud IAM Providers

| Dimension | OpenAI | AWS Bedrock / GCP Vertex |
|--------|--------|--------------------------|
| Authentication | API key | IAM / Service Accounts |
| Identity Context | Shared | Per-role / per-user |
| Native RBAC | ❌  | ✅  |
| Key Management Risk | High | Low |
| Telemetry Quality | Strong (tokens) | Strong (infra + tokens) |

**Conclusion**
OpenAI requires **additional governance layers** (API gateways, identity proxies) to meet enterprise security and compliance standards.


In [ ]:
# Run your Week 1 notebook with benchmark prompts
benchmark_prompts = [
    "Explain quantum computing in simple terms.",
    "Write a Python function to merge sorted lists.",
    "Summarize the last 5 years of AI research.",
    # Add 7-17 more varied prompts
]

# Collect telemetry
results = []
for prompt in benchmark_prompts:
    # AWS Bedrock
    bedrock_result = invoke_llama_with_governance(prompt, MODEL_ID)
    
    # OpenAI
    openai_result = invoke_openai_with_governance(prompt, model="gpt-4o")
    
    results.append({
        "prompt": prompt,
        "bedrock_latency_ms": bedrock_result["telemetry"].latency_ms,
        "bedrock_input_tokens": bedrock_result["telemetry"].input_tokens,
        "bedrock_output_tokens": bedrock_result["telemetry"].output_tokens,
        "openai_latency_ms": openai_result["telemetry"].latency_ms,
        "openai_input_tokens": openai_result["telemetry"].input_tokens,
        "openai_output_tokens": openai_result["telemetry"].output_tokens,
    })

# Calculate costs
df = pd.DataFrame(results)
df["bedrock_cost"] = (df["bedrock_input_tokens"] / 1000 * 0.003) + \
                     (df["bedrock_output_tokens"] / 1000 * 0.015)
df["openai_cost"] = (df["openai_input_tokens"] / 1000 * 0.005) + \
                    (df["openai_output_tokens"] / 1000 * 0.015)

# Summary statistics
print(df.groupby("provider").agg({
    "latency_ms": ["mean", "median", "std"],
    "cost": "sum"
}))

✓ Model Invocation Successful
  Latency: 7061.64ms
  Response Length: 2892 characters
✓ OpenAI Invocation Successful (gpt-4o)
✓ Model Invocation Successful
  Latency: 7085.45ms
  Response Length: 2310 characters
✓ OpenAI Invocation Successful (gpt-4o)
✓ Model Invocation Successful
  Latency: 6259.55ms
  Response Length: 2358 characters
✓ OpenAI Invocation Successful (gpt-4o)


KeyError: 'provider'

In [26]:
df

,prompt,bedrock_latency_ms,bedrock_input_tokens,bedrock_output_tokens,openai_latency_ms,openai_input_tokens,openai_output_tokens,bedrock_cost,openai_cost
0,Explain quantum computing in simple terms.,7061.64,8,512,5357.69,14,321,0.007704,0.004885
1,Write a Python function to merge sorted lists.,7085.45,9,512,6277.78,16,454,0.007707,0.006890
2,Summarize the last 5 years of AI research.,6259.55,12,436,7939.35,19,637,0.006576,0.009650


In [27]:
# Run your Week 1 notebook with benchmark prompts
benchmark_prompts = [
    "Explain quantum computing in simple terms.",
    "Write a Python function to merge sorted lists.",
    "Summarize the last 5 years of AI research.",
    "What are the benefits of cloud computing?",
]

# Collect telemetry
results = []
for prompt in benchmark_prompts:
    print(f"\n{'='*60}")
    print(f"Testing: {prompt[:50]}...")
    print('='*60)
    
    # AWS Bedrock
    bedrock_result = invoke_llama_with_governance(prompt, MODEL_ID)
    
    # OpenAI
    openai_result = invoke_openai_with_governance(prompt, model="gpt-4o")
    
    # Add Bedrock result
    results.append({
        "prompt": prompt,
        "provider": "bedrock",
        "latency_ms": bedrock_result["telemetry"].latency_ms,
        "input_tokens": bedrock_result["telemetry"].input_tokens,
        "output_tokens": bedrock_result["telemetry"].output_tokens,
        "success": bedrock_result["telemetry"].success,
    })
    
    # Add OpenAI result
    results.append({
        "prompt": prompt,
        "provider": "openai",
        "latency_ms": openai_result["telemetry"].latency_ms,
        "input_tokens": openai_result["telemetry"].input_tokens,
        "output_tokens": openai_result["telemetry"].output_tokens,
        "success": openai_result["telemetry"].success,
    })

# Create DataFrame
df = pd.DataFrame(results)

# Calculate costs (based on your research report pricing)
# Bedrock (Meta Llama 3): $0.003/1K input, $0.015/1K output
# OpenAI GPT-4o: $0.005/1K input, $0.015/1K output
df["cost"] = df.apply(
    lambda row: (
        (row["input_tokens"] / 1000 * 0.003) + (row["output_tokens"] / 1000 * 0.015)
        if row["provider"] == "bedrock"
        else (row["input_tokens"] / 1000 * 0.005) + (row["output_tokens"] / 1000 * 0.015)
    ),
    axis=1
)

# Summary statistics
print("\n" + "="*80)
print("EMPIRICAL BENCHMARKING RESULTS - SUMMARY STATISTICS")
print("="*80)

summary = df.groupby("provider").agg({
    "latency_ms": ["mean", "median", "std", "min", "max"],
    "input_tokens": ["mean", "median"],
    "output_tokens": ["mean", "median"],
    "cost": ["sum", "mean"],
    "success": lambda x: f"{(x.sum() / len(x)) * 100:.1f}%"
})

print(summary)

# Detailed comparison
print("\n" + "="*80)
print("COST COMPARISON")
print("="*80)

cost_comparison = df.groupby("provider")["cost"].agg(["sum", "mean", "median"])
cost_comparison["total_tokens"] = df.groupby("provider").apply(
    lambda x: (x["input_tokens"] + x["output_tokens"]).sum()
)
cost_comparison["cost_per_1k_tokens"] = (cost_comparison["sum"] / cost_comparison["total_tokens"]) * 1000

print(cost_comparison)

# Calculate savings
bedrock_total = cost_comparison.loc["bedrock", "sum"]
openai_total = cost_comparison.loc["openai", "sum"]
savings_pct = ((openai_total - bedrock_total) / openai_total) * 100

print(f"\n💰 Cost Savings: {savings_pct:.1f}% savings using Bedrock vs OpenAI")
print(f"   Bedrock Total: ${bedrock_total:.4f}")
print(f"   OpenAI Total: ${openai_total:.4f}")

# Latency comparison
print("\n" + "="*80)
print("LATENCY COMPARISON")
print("="*80)

latency_comparison = df.groupby("provider")["latency_ms"].agg(["mean", "median", "std"])
print(latency_comparison)

bedrock_latency = latency_comparison.loc["bedrock", "mean"]
openai_latency = latency_comparison.loc["openai", "mean"]
speed_diff = ((openai_latency - bedrock_latency) / openai_latency) * 100

if bedrock_latency < openai_latency:
    print(f"\n⚡ Speed: Bedrock is {abs(speed_diff):.1f}% faster than OpenAI")
else:
    print(f"\n⚡ Speed: OpenAI is {abs(speed_diff):.1f}% faster than Bedrock")

# Export results
df.to_csv("empirical_benchmarking_results.csv", index=False)
print("\n✅ Results saved to 'empirical_benchmarking_results.csv'")

# Display first few rows
print("\n" + "="*80)
print("SAMPLE RESULTS (First 5 prompts)")
print("="*80)
print(df[["prompt", "provider", "latency_ms", "input_tokens", "output_tokens", "cost"]].head(10))


Testing: Explain quantum computing in simple terms....
✓ Model Invocation Successful
  Latency: 7294.49ms
  Response Length: 2659 characters
✓ OpenAI Invocation Successful (gpt-4o)

Testing: Write a Python function to merge sorted lists....
✓ Model Invocation Successful
  Latency: 7227.04ms
  Response Length: 1829 characters
✓ OpenAI Invocation Successful (gpt-4o)

Testing: Summarize the last 5 years of AI research....
✓ Model Invocation Successful
  Latency: 7156.96ms
  Response Length: 2814 characters
✓ OpenAI Invocation Successful (gpt-4o)

Testing: What are the benefits of cloud computing?...
✓ Model Invocation Successful
  Latency: 6564.28ms
  Response Length: 2164 characters
✓ OpenAI Invocation Successful (gpt-4o)

EMPIRICAL BENCHMARKING RESULTS - SUMMARY STATISTICS
         latency_ms                                          input_tokens  \
               mean   median          std      min       max         mean   
provider                                                      

C:\Users\Musharraf\AppData\Local\Temp\ipykernel_20856\3102612306.py:78: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cost_comparison["total_tokens"] = df.groupby("provider").apply(
